In [1]:
cd ../home/ch1/my-project

# 1.4 Merging Branches

## In practice:
In this section, we'll integrate work from `new-branch` back into `main`, exploring different merge strategies.

## Types of Merges
Understanding the different types of merges in Git is crucial for effective branch management.

### Fast-forward merge
This merge strategy is used when the target branch can be updated to point to the commit of the source branch without needing to create a new commit.

### Three-way merge
A three-way merge is used when the two branches have diverged and a new commit is required to reconcile the differences.

## Discussion: How do different merges work?

### Fast-forward merge
To merge two branches, we use `git merge <branch-name>` while in the branch we want to merge into. For a fast-forward merge:

In [2]:
git switch main
git merge new-branch
git status
git log

Switched to branch 'main'
Updating 1fc9e27..77ce820
Fast-forward
 file1.txt | 4 +++-
 1 file changed, 3 insertions(+), 1 deletion(-)
On branch main
nothing to commit, working tree clean
commit 77ce820951d11ec67e06be7fde6354eadaf26dd8 (HEAD -> main, new-branch)
Author: mcallara <mcallara@gmail.com>
Date:   Thu Feb 8 23:34:45 2024 +0100

    Add third line to file1.txt

commit 1fc9e27026ba4f94c6a24d4970363411344f2ee7
Author: mcallara <mcallara@gmail.com>
Date:   Thu Feb 8 23:34:32 2024 +0100

    Add file1.txt


Updating 1fc9e27..77ce820


Fast-forward


 file1.txt | 4 +++-


 1 file changed, 3 insertions(+), 1 deletion(-)


...

    Add third line to file1.txt


commit 1fc9e27026ba4f94c6a24d4970363411344f2ee7


Author: mcallara <mcallara@gmail.com>


Date:   Thu Feb 8 23:34:32 2024 +0100


    Add file1.txt


```{admonition} What to notice
:class: hint
The `HEAD` of `main` is now pointing to the same commit as `new-branch`.
```

Switching branches may change the working directory, especially if they point to different commits.

## Exercise: Switching and Committing

In [3]:
git switch new-branch
touch file2.txt
echo -n "This is file 2." >> file2.txt
echo -en "\nThis is the fourth line." >> file1.txt
git add .
git commit -m "Add file2 and modify file1"
git switch main

Switched to branch 'new-branch'
[new-branch f9b0c6f] Add file2 and modify file1
 2 files changed, 3 insertions(+), 1 deletion(-)
 create mode 100644 file2.txt
Switched to branch 'main'


[new-branch f9b0c6f] Add file2 and modify file1


 2 files changed, 3 insertions(+), 1 deletion(-)


 create mode 100644 file2.txt


Switched to branch 'main'


```{admonition} What to notice
:class: hint
The working directory content changes when switching branches.
```

Git prevents loss of work by disallowing branch switching with uncommitted changes.

## Exercise: Preventing Data Loss

In [4]:
git switch new-branch
echo -en "\nThis is the fifth line." >> file1.txt
git switch main

Switched to branch 'new-branch'
error: Your local changes to the following files would be overwritten by checkout:
	file1.txt
Please commit your changes or stash them before you switch branches.
Aborting


error: Your local changes to the following files would be overwritten by checkout:


	file1.txt


Please commit your changes or stash them before you switch branches.


Aborting


: 1

```{admonition} What to notice
:class: hint
Git will not allow branch switching with uncommitted changes to protect against data loss.
```

## Note on Staging
Changes made after staging are not automatically included in the staged snapshot.

## Exercise: Staging and Modifying

In [5]:
git add file1.txt
echo -en "\nThis is the sixth line." >> file1.txt
git add file1.txt
git commit -m "Add fifth and sixth lines to file1.txt"

[new-branch 812f652] Add fifth and sixth lines to file1.txt
 1 file changed, 3 insertions(+), 1 deletion(-)


 1 file changed, 3 insertions(+), 1 deletion(-)


## Viewing All Commits

In [6]:
git switch main
git log --all

Switched to branch 'main'
commit 812f652750efde1723ec1656194764a0ff0b8324 (new-branch)
Author: mcallara <mcallara@gmail.com>
Date:   Thu Feb 8 23:34:58 2024 +0100

    Add fifth and sixth lines to file1.txt

commit f9b0c6ff617d23cfc04251eec504dc8a8e8a4bb6
Author: mcallara <mcallara@gmail.com>
Date:   Thu Feb 8 23:34:57 2024 +0100

    Add file2 and modify file1

commit 77ce820951d11ec67e06be7fde6354eadaf26dd8 (HEAD -> main)
Author: mcallara <mcallara@gmail.com>
Date:   Thu Feb 8 23:34:45 2024 +0100

    Add third line to file1.txt

commit 1fc9e27026ba4f94c6a24d4970363411344f2ee7
Author: mcallara <mcallara@gmail.com>
Date:   Thu Feb 8 23:34:32 2024 +0100

    Add file1.txt


commit 812f652750efde1723ec1656194764a0ff0b8324 (new-branch)


Author: mcallara <mcallara@gmail.com>


Date:   Thu Feb 8 23:34:58 2024 +0100


    Add fifth and sixth lines to file1.txt


...

    Add third line to file1.txt


commit 1fc9e27026ba4f94c6a24d4970363411344f2ee7


Author: mcallara <mcallara@gmail.com>


Date:   Thu Feb 8 23:34:32 2024 +0100


    Add file1.txt


```{admonition} What to notice
:class: hint
Commits from `new-branch` are visible with `git log --all`, showing the comprehensive history across branches.
```

## Detached HEAD State
Exploring commits directly can lead to a detached HEAD state, which means you're no longer working on the tip of a branch.

## Creating and Switching Branches
Git offers shortcuts to create a new branch and switch to it in one operation, enhancing workflow efficiency.

In [7]:
git switch -c new-feature-branch

Switched to a new branch 'new-feature-branch'


This chapter provides practical exercises to understand merging strategies, branch management, and Git's protections against data loss.